# IMDB

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

import keras

from keras import backend as K
from keras import regularizers
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization, Masking, Activation

import matplotlib.pyplot as plt

In [3]:
def custom_loss(y_true,y_pred):
    y_mask=keras.backend.clip(y_true, 0, 0.01)*100
    return K.mean(K.square(y_mask*(y_pred - y_true)), axis=-1)


## Download the data

In [4]:
X_train = np.load("../data/preprocessed_data/X_train.npy", allow_pickle=True)
y_train = np.load("../data/preprocessed_data/y_train.npy", allow_pickle=True)
X_test = np.load("../data/preprocessed_data/X_test.npy", allow_pickle=True)
y_test = np.load("../data/preprocessed_data/y_test.npy", allow_pickle=True)
ratedlist = np.load("../data/preprocessed_data/ratedlist.npy", allow_pickle=True)

In [7]:
input_img = Input(shape=(X_train.shape[1],)) # All movies

encoded = Masking(mask_value=0)(input_img)

encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(50, activation='relu')(encoded)
encoded = Dense(12, activation='relu')(encoded)

decoded = Dense(50, activation='relu')(encoded)
decoded = Dense(12, activation='relu')(decoded)
decoded = Dense(y_train.shape[1], activation='sigmoid')(decoded) # All movies

autoencoder = Model(input_img, decoded)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 193609)]          0         
_________________________________________________________________
masking_2 (Masking)          (None, 193609)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                12391040  
_________________________________________________________________
dense_9 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_10 (Dense)             (None, 12)                612       
_________________________________________________________________
dense_11 (Dense)             (None, 50)                650       
_________________________________________________________________
dense_12 (Dense)             (None, 12)                612 

In [8]:
autoencoder.compile(loss=custom_loss, optimizer='adam')

In [9]:
split = int(X_train.shape[0] * 0.8)

In [10]:
epochs = 15
batchsize = 128

autoencoder.fit(X_train, y_train, epochs=epochs, batch_size=batchsize, validation_data=(X_test, y_test))
# autoencoder.fit(X_train[:split], y_train[:split], epochs=epochs, batch_size=batchsize, validation_data=(X_test[split:], y_test[split:]))

Epoch 1/15
5/5 [==============================] - 6s 582ms/step - loss: 1.8578e-04 - val_loss: 1.8574e-04
Epoch 2/15
5/5 [==============================] - 1s 198ms/step - loss: 1.8571e-04 - val_loss: 1.8566e-04
Epoch 3/15
5/5 [==============================] - 1s 189ms/step - loss: 1.8562e-04 - val_loss: 1.8555e-04
Epoch 4/15
5/5 [==============================] - 1s 191ms/step - loss: 1.8551e-04 - val_loss: 1.8544e-04
Epoch 5/15
5/5 [==============================] - 1s 177ms/step - loss: 1.8539e-04 - val_loss: 1.8531e-04
Epoch 6/15
5/5 [==============================] - 1s 177ms/step - loss: 1.8526e-04 - val_loss: 1.8517e-04
Epoch 7/15
5/5 [==============================] - 1s 182ms/step - loss: 1.8512e-04 - val_loss: 1.8502e-04
Epoch 8/15
5/5 [==============================] - 1s 187ms/step - loss: 1.8497e-04 - val_loss: 1.8487e-04
Epoch 9/15
5/5 [==============================] - 1s 186ms/step - loss: 1.8481e-04 - val_loss: 1.8471e-04
Epoch 10/15
5/5 [=============================

In [11]:
y_predict = autoencoder.predict(X_test)

In [12]:
SE = 0 # SquaredError

for r in range(ratedlist.shape[0]):
    
    amount_of_ratings = np.max(np.where(ratedlist[r] > 0)[0])
    
    diff = np.sum(
        (y_test[r, ratedlist[r]] * 5 - y_predict[r, ratedlist[r]] * 5) ** 2
    ) / amount_of_ratings
    SE += diff

In [13]:
RMSE = np.sqrt(SE / ratedlist.shape[0])
print(RMSE)

2.358106142082608


In [14]:
print("Actual ratings")
print(y_test[0, ratedlist[0]] * 5)
print("\nPredicted ratings")
print(y_predict[0, ratedlist[0]] * 5 )

Actual ratings
[0.16 0.16 0.16 0.2  0.2  0.12 0.2  0.16 0.2  0.2  0.2  0.2  0.12 0.2
 0.16 0.2  0.12 0.12 0.2  0.16 0.16 0.2  0.16 0.12 0.16 0.2  0.16 0.12
 0.2  0.16 0.16 0.2  0.16 0.16 0.16 0.2  0.2  0.12 0.2  0.12 0.16 0.12
 0.12 0.16 0.2  0.2  0.2  0.16 0.2  0.12 0.2  0.2  0.2  0.2  0.12 0.2
 0.2  0.16 0.2  0.16 0.2  0.2  0.2  0.16 0.2  0.2  0.16 0.2  0.2  0.2
 0.2  0.2  0.16 0.2  0.2  0.16 0.08 0.2  0.2  0.2  0.2  0.2  0.2  0.12
 0.16 0.2  0.2  0.2  0.2  0.2  0.2  0.16 0.12 0.12 0.12 0.12 0.16 0.16
 0.2  0.16 0.2  0.12 0.2  0.2  0.16 0.2  0.12 0.12 0.2  0.16 0.16 0.2
 0.16 0.16 0.2  0.2  0.16 0.16 0.2  0.16 0.2  0.16 0.2  0.16 0.2  0.16
 0.2  0.2  0.2  0.12 0.2  0.16 0.16 0.16 0.2  0.2  0.2  0.2  0.2  0.16
 0.2  0.16 0.16 0.08 0.16 0.16 0.2  0.2  0.08 0.2  0.16 0.2  0.08 0.2
 0.16 0.12 0.2  0.16 0.2  0.2  0.16 0.16 0.2  0.12 0.2  0.2  0.2  0.2
 0.2  0.16 0.08 0.16 0.16 0.2  0.16 0.16 0.2  0.12 0.2  0.2  0.2  0.2
 0.16 0.16 0.2  0.2  0.2  0.16 0.2  0.2  0.2  0.2  0.2  0.16 0.2  0.2

In [15]:
import os

In [16]:
os.mkdir('../snapshots')

In [17]:
autoencoder.save("../snapshots/imdb-model.h5")

## Upload this model to the cloud

## Make sure this file is executable in one run